In [ ]:
!pip install pandas numpy scikit-learn PyPDF2 catboost

In [ ]:
import pandas as pd
import numpy as np
from glob import glob
import os
import PyPDF2
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from catboost import CatBoostClassifier
from sklearn.multioutput import MultiOutputClassifier
import re
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Read the Excel file
df_o = pd.read_excel('/kaggle/input/MDA-Disclosure-Quality-Assessment/Training_dataset.xlsx')

# Replace the specified value in the 'Filename' column
df_o.loc[df_o['Filename'] == 'III_2022057225', 'Filename'] = 'III_2022129989'

#Rename column
new_column_names = ['Year', 'Quarter', 'Filename', 'Report_quality', 'ภาพรวม', 'สภาวะตลาด', 'สรุปการผลการดำเนินงาน', 'เหตุการณ์สำคัญ', 'ปัจจัยต่อการเติบโต', 'เปิดเผยรายได้', 'เปิดเผยกำไรที่เปลี่ยนไป', 'คำอธิบายรายได้', 'คำอธิบายกำไรที่เปลี่ยนไป', 'วิเคราะห์งบหรือฐานะทางการเงิน', 'รายงาน ESG']
df_o.columns = new_column_names

# Save the modified DataFrame to a new Excel file
df_o.to_excel('/kaggle/working/Training_dataset.xlsx', index=False)

In [ ]:
class MDAQualityAssessor:
    def __init__(self):
        # Enhanced keywords for better section detection
        self.section_keywords = {
            'ภาพรวม': ['ภาพรวม', 'ธุรกิจ', 'การดำเนินงาน', 'เศรษฐกิจ', 'อุตสาหกรรม', 'ลักษณะการประกอบธุรกิจ'],
            'สภาวะตลาด': ['ตลาด', 'การแข่งขัน', 'อุตสาหกรรม', 'คู่แข่ง', 'ส่วนแบ่งตลาด', 'ภาวะการแข่งขัน'],
            'สรุปการผลการดำเนินงาน': ['ผลการดำเนินงาน', 'กำไร', 'รายได้', 'ต้นทุน', 'ยอดขาย', 'การเติบโต'],
            'เหตุการณ์สำคัญ': ['เหตุการณ์', 'พัฒนาการ', 'ความคืบหน้า', 'การเปลี่ยนแปลง', 'ผลกระทบ'],
            'ปัจจัยต่อการเติบโต': ['ปัจจัย', 'การเติบโต', 'อนาคต', 'แผนการ', 'โอกาส', 'ความเสี่ยง'],
            'วิเคราะห์งบหรือฐานะทางการเงิน': ['งบการเงิน', 'ฐานะการเงิน', 'สินทรัพย์', 'หนี้สิน', 'สภาพคล่อง', 'อัตราส่วนทางการเงิน'],
            'รายงาน ESG': ['ESG', 'สิ่งแวดล้อม', 'สังคม', 'ธรรมาภิบาล', 'ความยั่งยืน', 'พลังงาน']
        }
        
        # Revenue and profit explanation patterns
        self.financial_patterns = {
            'revenue_explanation': [
                r'รายได้.{0,50}เพิ่มขึ้น|ลดลง.{0,100}เนื่องจาก',
                r'รายได้.{0,50}เปลี่ยนแปลง.{0,100}สาเหตุ',
                r'การเปลี่ยนแปลงของรายได้.{0,100}จาก'
            ],
            'profit_explanation': [
                r'กำไร.{0,50}เพิ่มขึ้น|ลดลง.{0,100}เนื่องจาก',
                r'กำไร.{0,50}เปลี่ยนแปลง.{0,100}สาเหตุ',
                r'การเปลี่ยนแปลงของกำไร.{0,100}จาก'
            ]
        }
        
        self.label_encoders = {}
        self.model = None
        self.tfidf_vectorizer = None

    def extract_text_from_pdf(self, pdf_path):
        """Extract text from PDF file with enhanced error handling"""
        try:
            if not os.path.exists(pdf_path):
                return ""
                
            text = ""
            with open(pdf_path, 'rb') as file:
                reader = PyPDF2.PdfReader(file)
                for page in reader.pages:
                    text += page.extract_text()
            
            # Basic text cleaning
            text = re.sub(r'\s+', ' ', text)
            text = text.lower().strip()
            return text
        except Exception as e:
            print(f"Error processing {pdf_path}: {str(e)}")
            return ""

    def create_features(self, text):
        """Create enhanced features from text"""
        features = {}
        
        # Keyword-based features
        for section, keywords in self.section_keywords.items():
            count = sum(1 for keyword in keywords if keyword.lower() in text.lower())
            features[f"{section}_keyword_count"] = count
            
        # Financial explanation features
        for pattern_type, patterns in self.financial_patterns.items():
            matches = sum(1 for pattern in patterns if re.search(pattern, text))
            features[f"{pattern_type}_count"] = matches
        
        # Additional features
        features.update({
            'total_length': len(text),
            'num_paragraphs': len(text.split('\n\n')),
            'num_numbers': len(re.findall(r'\d+(?:\.\d+)?', text)),
            'num_percentages': len(re.findall(r'\d+(?:\.\d+)?%', text)),
            'has_tables': 1 if len(re.findall(r'\|\s*\w+\s*\|', text)) > 0 else 0
        })
        
        return features

    def process_training_data(self, excel_path, pdf_base_path):
        """Process training data with enhanced feature extraction"""
        df = pd.read_excel(excel_path, sheet_name='Sheet1')
        
        all_features = []
        all_texts = []
        valid_indices = []
        
        for idx, row in df.iterrows():
            pdf_path = os.path.join(pdf_base_path, f"{row['Year']}_Q{row['Quarter']}", f"{row['Filename']}.pdf")
            text = self.extract_text_from_pdf(pdf_path)
            
            if text:
                all_texts.append(text)
                features = self.create_features(text)
                all_features.append(features)
                valid_indices.append(idx)
        
        df = df.iloc[valid_indices]
        
        # Enhanced TF-IDF features
        self.tfidf_vectorizer = TfidfVectorizer(
            max_features=1500,
            ngram_range=(1, 2),
            stop_words='english'
        )
        tfidf_features = self.tfidf_vectorizer.fit_transform(all_texts)
        
        features_df = pd.DataFrame(all_features)
        
        # Prepare labels with updated columns
        label_columns = [
            'Report_quality', 'ภาพรวม', 'สภาวะตลาด', 'สรุปการผลการดำเนินงาน',
            'เหตุการณ์สำคัญ', 'ปัจจัยต่อการเติบโต', 'เปิดเผยรายได้',
            'เปิดเผยกำไรที่เปลี่ยนไป', 'คำอธิบายรายได้', 'คำอธิบายกำไรที่เปลี่ยนไป',
            'วิเคราะห์งบหรือฐานะทางการเงิน', 'รายงาน ESG'
        ]
        
        # Encode labels
        encoded_labels = {}
        for col in label_columns:
            le = LabelEncoder()
            encoded_labels[col] = le.fit_transform(df[col].fillna('N'))
            self.label_encoders[col] = le
        
        combined_features = np.hstack([tfidf_features.toarray(), features_df.values])
        
        return combined_features, pd.DataFrame(encoded_labels)

    def train_model(self, features, labels):
        """Train model using CatBoost"""
        base_classifier = CatBoostClassifier(
            iterations=500,
            learning_rate=0.1,
            depth=6,
            loss_function='MultiClass',
            verbose=False
        )
        self.model = MultiOutputClassifier(base_classifier)
        self.model.fit(features, labels.values)
#####
    def predict(self, pdf_path):
        """Make predictions with enhanced error handling"""
        try:
            text = self.extract_text_from_pdf(pdf_path)
            if not text:
                return self._get_default_predictions()
                
            features = self.create_features(text)
            text_features = self.tfidf_vectorizer.transform([text])
            combined_features = np.hstack([text_features.toarray(), pd.DataFrame([features]).values])
            
            # Make predictions with the model
            predictions = self.model.predict(combined_features)
            
            # Flatten the prediction results
            predictions = np.array(predictions).reshape(-1)
            
            # Map predictions back to labels
            results = {}
            for i, (col, le) in enumerate(self.label_encoders.items()):
                results[col] = le.inverse_transform([predictions[i]])[0]
                
            return results
        except Exception as e:
            print(f"Error making prediction for {pdf_path}: {str(e)}")
            return self._get_default_predictions()
#####
    def _get_default_predictions(self):
        """Return default predictions with updated columns"""
        return {
            'Report_quality': 'C',
            'ภาพรวม': 'N',
            'สภาวะตลาด': 'N',
            'สรุปการผลการดำเนินงาน': 'N',
            'เหตุการณ์สำคัญ': 'N',
            'ปัจจัยต่อการเติบโต': 'N',
            'เปิดเผยรายได้': 'N',
            'เปิดเผยกำไรที่เปลี่ยนไป': 'N',
            'คำอธิบายรายได้': 'N',
            'คำอธิบายกำไรที่เปลี่ยนไป': 'N',
            'วิเคราะห์งบหรือฐานะทางการเงิน': 'N',
            'รายงาน ESG': 'N'
        }

    def create_submission(self, test_folder, sample_submission_path, output_path):
        """Create submission file with sample submission matching"""
        results = []
        pdf_files = glob(os.path.join(test_folder, '*.pdf'))
        
        for pdf_file in pdf_files:
            filename = os.path.basename(pdf_file).replace('.pdf', '')
            predictions = self.predict(pdf_file)
            results.append({'Filename': filename, **predictions})
        
        submission_df = pd.DataFrame(results)
        
        # Match with sample submission
        sample_submission = pd.read_csv(sample_submission_path)
        merged_df = pd.merge(sample_submission[['Filename']], submission_df, on='Filename', how='outer')
        
        # Fill missing values
        merged_df = merged_df.fillna(method='bfill').fillna('N')
        
        # Ensure correct column order
        required_columns = [
            'Filename', 'Report_quality', 'ภาพรวม', 'สภาวะตลาด',
            'สรุปการผลการดำเนินงาน', 'เหตุการณ์สำคัญ', 'ปัจจัยต่อการเติบโต',
            'เปิดเผยรายได้', 'เปิดเผยกำไรที่เปลี่ยนไป', 'คำอธิบายรายได้',
            'คำอธิบายกำไรที่เปลี่ยนไป', 'วิเคราะห์งบหรือฐานะทางการเงิน', 'รายงาน ESG'
        ]
        merged_df = merged_df[required_columns]
        
        merged_df.to_csv(output_path, index=False)
        print(f"Submission file saved to {output_path}")

def main():
    assessor = MDAQualityAssessor()
    
    # Paths
    train_excel = '/kaggle/working/Training_dataset.xlsx'
    train_base_path = '/kaggle/input/MDA-Disclosure-Quality-Assessment/train'
    test_folder = '/kaggle/input/MDA-Disclosure-Quality-Assessment/test/2023_Q4'
    sample_submission_path = '/kaggle/input/MDA-Disclosure-Quality-Assessment/sample_submission.csv'
    submission_path = '/kaggle/working/submission_CatBoot.csv'
    
    try:
        print("Processing training data...")
        features, labels = assessor.process_training_data(train_excel, train_base_path)
        
        print("Training model...")
        assessor.train_model(features, labels)
        
        print("Creating submission file...")
        assessor.create_submission(test_folder, sample_submission_path, submission_path)
        
    except Exception as e:
        print(f"Error in main execution: {str(e)}")

if __name__ == "__main__":
    main()